<a href="https://colab.research.google.com/github/Spili11/DS_Praktikum/blob/main/DNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 9.0MB/s 


In [2]:

import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np
import gzip
import matplotlib.pyplot as plt
from math import floor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
classes = ['notpresent', 'present']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://github.com/simecek/dspracticum2020/raw/master/lecture_08/assignment/g4/g4_{c}_{s}.fa.gz", f"g4_{c}_{s}.fa.gz")

for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

for c in classes:
    for s in sets:
        with gzip.open(f"g4_{c}_{s}.fa.gz", "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])

In [4]:
batch_size = 128

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 210000 files belonging to 2 classes.


In [5]:
def getKmer(sequence, size = 5):
  sequence = sequence[2: -1]
  sequence = sequence.replace(' ', '')
  length = len(sequence)
  final = []
  for i in range(floor(length/ size )):
    final += [sequence[(size * i):(size*(i +1))]]
  return final



In [6]:
Y = []
X = []
for seq_batch, label_batch in raw_train_ds.take(-1):
  for i in range(len(seq_batch)):
    Y += [label_batch.numpy()[i]]
    X += [getKmer(str(seq_batch.numpy()[i]))]  
 

In [7]:
X_train = X.copy()
for j in range(len(X)):
  X_train[j] = " ".join(X[j])

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(ngram_range=(4,4))
X_train_final=cv.fit_transform(X_train)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_final, 
                                                   Y, 
                                                    test_size = 0.20, 
                                                    random_state=42)

In [10]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [11]:
y_pred = classifier.predict(X_test)

In [12]:
import pandas as pd
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))

Predicted     0      1
Actual                
0          8331  12622
1          3131  17916
